In [162]:
import torch
from utils import process_dataset
from argparse import ArgumentParser
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import numpy as np
import random

seed = 123
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [163]:
# datasets = "../datasets/pondlet_gloss_HSK3_2021-all.csv;../datasets/pondlet_STB_HSK3_2021-all.csv"
datasets = "../datasets/pondlet_STB_HSK3_20220429_new.csv"
balance_data = process_dataset(datasets)
# balance_data.drop("ID", inplace=True, axis=1)
# balance_data.drop("None", inplace=True, axis=1)
# balance_data.drop("7-9", inplace=True, axis=1)

datasets ../datasets/pondlet_STB_HSK3_20220429_new.csv
len-balance_data(one) 1616


In [164]:
balance_data.head()

,Label,ID,Length,1,2,3,4,5,6,7-9,None
0,Lv.1,Pondlet_0452_001917,100,48,3,10,0,0,0,8,1
1,Lv.4,Pondlet_0642_013141,314,107,39,12,13,5,4,22,0
2,Lv.1,Pondlet_0197_003167,38,24,0,0,0,0,0,0,0
3,Lv.6,Pondlet_0726_015179,610,166,63,58,36,12,27,24,5
4,Lv.3,Pondlet_0722_013593,253,85,30,6,4,9,2,4,8


In [165]:
level = 6
balance_data = balance_data.loc[balance_data['Label'] == f"Lv.{level}"]
balance_data.head()

,Label,ID,Length,1,2,3,4,5,6,7-9,None
3,Lv.6,Pondlet_0726_015179,610,166,63,58,36,12,27,24,5
10,Lv.6,Pondlet_0826_020259,586,161,58,48,20,30,12,25,5
25,Lv.6,Pondlet_0891_020413,518,144,50,53,23,13,22,43,3
31,Lv.6,Pondlet_0721_015177,596,152,53,58,35,18,15,19,13
33,Lv.6,Pondlet_0760_019610,594,109,49,53,54,37,13,15,10


In [166]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [167]:
class Model(nn.Module):

    def __init__(self, input_dim, output_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 8)
        self.layer2 = nn.Linear(8, 16)
        self.layer3 = nn.Linear(16, 24)
        self.layer4 = nn.Linear(24, 32)
        self.layer5 = nn.Linear(32, 64)
        self.layer6 = nn.Linear(64, 32)
        self.layer7 = nn.Linear(32, 24)
        self.layer8 = nn.Linear(24, 16)
        self.layer9 = nn.Linear(16, output_dim)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = F.relu(self.layer5(x))
        x = F.relu(self.layer6(x))
        x = F.relu(self.layer7(x))
        x = F.relu(self.layer8(x))
        return self.layer9(x)


In [168]:
model = Model(9, 6)
model.load_state_dict(torch.load("model_fold1.pt"))
model.eval()

Model(
  (layer1): Linear(in_features=9, out_features=8, bias=True)
  (layer2): Linear(in_features=8, out_features=16, bias=True)
  (layer3): Linear(in_features=16, out_features=24, bias=True)
  (layer4): Linear(in_features=24, out_features=32, bias=True)
  (layer5): Linear(in_features=32, out_features=64, bias=True)
  (layer6): Linear(in_features=64, out_features=32, bias=True)
  (layer7): Linear(in_features=32, out_features=24, bias=True)
  (layer8): Linear(in_features=24, out_features=16, bias=True)
  (layer9): Linear(in_features=16, out_features=6, bias=True)
)

In [169]:
predict_labels = []

for i, row in balance_data.iterrows():
    x = torch.FloatTensor(row[2:].tolist())
    val, index = torch.max(model(x), 0)
    predict_labels.append(f"Lv.{index.item() + 1}")



In [170]:
balance_data["Predict_Label"] = predict_labels

In [171]:
balance_data.head()

,Label,ID,Length,1,2,3,4,5,6,7-9,None,Predict_Label
3,Lv.6,Pondlet_0726_015179,610,166,63,58,36,12,27,24,5,Lv.6
10,Lv.6,Pondlet_0826_020259,586,161,58,48,20,30,12,25,5,Lv.6
25,Lv.6,Pondlet_0891_020413,518,144,50,53,23,13,22,43,3,Lv.6
31,Lv.6,Pondlet_0721_015177,596,152,53,58,35,18,15,19,13,Lv.6
33,Lv.6,Pondlet_0760_019610,594,109,49,53,54,37,13,15,10,Lv.6


In [172]:
cols = ['ID', 'Length', '1', '2', '3', '4', '5', '6', '7-9', 'None', 'Label', 'Predict_Label']

In [173]:
balance_data = balance_data[cols]
balance_data.head()

,ID,Length,1,2,3,4,5,6,7-9,None,Label,Predict_Label
3,Pondlet_0726_015179,610,166,63,58,36,12,27,24,5,Lv.6,Lv.6
10,Pondlet_0826_020259,586,161,58,48,20,30,12,25,5,Lv.6,Lv.6
25,Pondlet_0891_020413,518,144,50,53,23,13,22,43,3,Lv.6,Lv.6
31,Pondlet_0721_015177,596,152,53,58,35,18,15,19,13,Lv.6,Lv.6
33,Pondlet_0760_019610,594,109,49,53,54,37,13,15,10,Lv.6,Lv.6


In [174]:
equal_data = balance_data.loc[balance_data['Label'] == balance_data['Predict_Label']]
equal_data.to_csv(f"pondlet_hsk3_predict_diff_lv{level}_label相同.csv", index=False)

In [175]:
unequal_data = balance_data.loc[balance_data['Label'] != balance_data['Predict_Label']]
unequal_data.to_csv(f"pondlet_hsk3_predict_diff_lv{level}_label不同.csv", index=False)